# ERA5 plots and animatinos

In [1]:
import xarray as xr
from sithom.plot import plot_defaults
from sithom.xr import plot_units
from src.constants import IBTRACS_NC
plot_defaults()

In [2]:
ds = xr.open_dataset(IBTRACS_NC)

In [12]:
ds

<xarray.Dataset>
Dimensions:           (storm: 13612, date_time: 360, quadrant: 4)
Coordinates:
    time              (storm, date_time) datetime64[ns] ...
    lat               (storm, date_time) float32 ...
    lon               (storm, date_time) float32 ...
Dimensions without coordinates: storm, date_time, quadrant
Data variables: (12/147)
    numobs            (storm) float32 65.0 25.0 49.0 89.0 ... 33.0 31.0 33.0
    sid               (storm) |S13 b'1842298N11080' ... b'2022145N10267'
    season            (storm) float32 1.842e+03 1.845e+03 ... 2.022e+03
    number            (storm) int16 1 1 1 2 3 4 5 6 ... 21 22 23 24 25 26 27 28
    basin             (storm, date_time) |S2 ...
    subbasin          (storm, date_time) |S2 ...
    ...                ...
    reunion_gust      (storm, date_time) float32 ...
    reunion_gust_per  (storm, date_time) float32 ...
    usa_seahgt        (storm, date_time) float32 ...
    usa_searad        (storm, date_time, quadrant) float32 ...
    storm_speed       (storm, date_time) float32 ...
    storm_dir         (storm, date_time) float32 ...
Attributes: (12/50)
    title:                      IBTrACS - International Best Track Archive fo...
    summary:                    The intent of the IBTrACS project is to overc...
    source:                     The original data are tropical cyclone positi...
    Conventions:                ACDD-1.3
    Conventions_note:           Data are nearly CF-1.7 compliant. The sole is...
    product_version:            v04r00
    ...                         ...
    license:                    These data may be redistributed and used with...
    featureType:                trajectory
    cdm_data_type:              Trajectory
    comment:                    The tracks of TCs generally look like a traje...
    nco_openmp_thread_number:   1
    NCO:                        4.4.3

In [13]:
ds["basin"]

<xarray.DataArray 'basin' (storm: 13612, date_time: 360)>
[4900320 values with dtype=|S2]
Coordinates:
    time     (storm, date_time) datetime64[ns] ...
    lat      (storm, date_time) float32 ...
    lon      (storm, date_time) float32 ...
Dimensions without coordinates: storm, date_time
Attributes:
    long_name:              Current basin
    Note:                   EP=East_Pacific NA=North_Atlantic NI=North_Indian...
    coverage_content_type:  thematicClassification

In [6]:
for i in ds.storm:
    print(ds.sel(storm=i).name.values)

b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'
b'NOT_NAMED'

KeyboardInterrupt: 

In [11]:
ds.basin.plot()

<xarray.DataArray 'basin' (storm: 13612, date_time: 360)>
[4900320 values with dtype=|S2]
Coordinates:
    time     (storm, date_time) datetime64[ns] ...
    lat      (storm, date_time) float32 ...
    lon      (storm, date_time) float32 ...
Dimensions without coordinates: storm, date_time
Attributes:
    long_name:              Current basin
    Note:                   EP=East_Pacific NA=North_Atlantic NI=North_Indian...
    coverage_content_type:  thematicClassification

In [30]:
plot_units(ds)

<xarray.Dataset>
Dimensions:    (longitude: 81, latitude: 81, time: 288)
Coordinates:
  * longitude  (longitude) float32 -100.0 -99.75 -99.5 ... -80.5 -80.25 -80.0
  * latitude   (latitude) float32 35.0 34.75 34.5 34.25 ... 15.5 15.25 15.0
  * time       (time) datetime64[ns] 2005-08-20 ... 2005-08-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
    mwd        (time, latitude, longitude) float32 ...
    mwp        (time, latitude, longitude) float32 ...
    sst        (time, latitude, longitude) float32 ...
    swh        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-05-24 18:12:03 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

: 

ds.swh.sel(time="2005-08-2T01:00:00").plot()
ds.tp.sel(time="2005-08-2T01:00:00").plot()